# Common import statements

In [1]:
import pandas as pd
from pandas import DataFrame as df
import IPython as ip

import seaborn as sns
sns.set(color_codes=True)

import matplotlib.pyplot as plt
# Set input dir
import sys
sys.path.append("/home/mschlupp/pythonTools")
files_dir = "files/"

## Split data into `hasEvents` and `noEvents`

In [8]:
ga_full = pd.read_csv(files_dir+"traintest_fullevt.csv")

In [10]:
ga_hasEvts = ga_full[ga_full.hasEvents>0]

In [9]:
ga_noEvts = ga_full[ga_full.hasEvents==0]

In [13]:
ga_noEvts = ga_noEvts.drop(["hasEvents", "nEvts", "longitude_mean"
                , "longitude_variance", "latitude_mean"
                , "latitude_variance", "usageTime_mean"
                , "usageTime_variance", "usageDay_mean"
                ,"usageDay_variance"], axis=1)

In [14]:
ga_noEvts.to_csv("files/ga_noEvts.csv", index=False)

In [15]:
ga_hasEvts.to_csv("files/ga_hasEvts.csv", index=False)

In [17]:
%ls files/

app_events.csv              label_categories.csv
app_labels.csv              phone_brand_device_model.csv
apps_labels_categories.csv  phone_brand_device_model_engl.csv
events.csv                  sample_submission.csv
events_day_hour.csv         traintest_fullevt.csv
ga_hasEvts.csv              traintest_phone.csv
ga_noEvts.csv               traintest_phone_day_hour.csv
gender_age_test.csv         traintest_phone_evts.csv
gender_age_train.csv


## Split events data in ```hasApp``` and ```noApp```

In [ ]:
evts = pd.read_csv("files/events_day_hour.csv", index_col=0)

In [ ]:
evts["isTrain"] = [ga_full[ga_full.device_id==x].isTrain for x in evts.device_id]

In [ ]:
evts.head(2)